<a href="https://colab.research.google.com/github/JeongJeong-code/P7_scoring/blob/main/exploration_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import missingno as mi
from IPython.core.interactiveshell import InteractiveShell
from google.colab import drive
InteractiveShell.ast_node_interactivity = "all"
pd.option_context('display.max_rows', None, 'display.max_columns', None)

In [10]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
path =r'/content/drive/MyDrive/P7_scoring/'
df = pd.read_csv(path + 'df_processed.csv')

In [12]:
column_list = list(df.select_dtypes(exclude=object).columns)

In [13]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]

In [14]:
array_inf_nb

array([], shape=(1, 0), dtype=float64)

In [49]:

nan_columns = pd.Series(train_df.columns[np.where(np.isnan(train_df).sum()>0)])
inf_columns = pd.Series(train_df.columns[np.where(np.isinf(train_df).sum()>0)])
serie_inf_nb = pd.Series(np.empty(len(inf_columns)))
serie_inf_ratio = pd.Series(np.empty(len(inf_columns)))
serie_nan_ratio = pd.Series(np.empty(len(nan_columns)))
serie_nan_nb = pd.Series(np.empty(len(nan_columns)))

for i in range(len(nan_columns)):
    serie_nan_nb[i] = np.isnan(train_df.loc[:,nan_columns[i]]).sum()
    serie_nan_ratio[i] = np.isnan(train_df.loc[:,nan_columns[i]]).sum()/len(train_df)

nan_df = pd.concat([nan_columns,serie_nan_nb,serie_nan_ratio],axis=1)
nan_df.rename(columns = {0:'nan_columns',1:'nan_nb',2:'nan_ratio'} )

for i in range(len(inf_columns)):
    serie_inf_nb[i] = np.isinf(train_df.loc[:,inf_columns[i]]).sum()
    serie_inf_ratio[i] = np.isinf(train_df.loc[:,inf_columns[i]]).sum()/len(train_df)

inf_df = pd.concat([inf_columns,serie_inf_nb,serie_inf_ratio],axis=1)
inf_df.rename(columns = {0:'inf_columns',1:'inf_nb',2:'inf_ratio'} )

,nan_columns,nan_nb,nan_ratio
0,AMT_ANNUITY,12.0,0.000039
1,AMT_GOODS_PRICE,278.0,0.000904
2,DAYS_EMPLOYED,55374.0,0.180074
3,OWN_CAR_AGE,202927.0,0.659910
4,CNT_FAM_MEMBERS,2.0,0.000007
...,...,...,...
611,CC_NAME_CONTRACT_STATUS_nan_MAX,220602.0,0.717389
612,CC_NAME_CONTRACT_STATUS_nan_MEAN,220602.0,0.717389
613,CC_NAME_CONTRACT_STATUS_nan_SUM,220602.0,0.717389
614,CC_NAME_CONTRACT_STATUS_nan_VAR,221234.0,0.719444


,inf_columns,inf_nb,inf_ratio
0,PREV_APP_CREDIT_PERC_MAX,1.0,0.000003
1,REFUSED_APP_CREDIT_PERC_MAX,1.0,0.000003
2,INSTAL_PAYMENT_PERC_MAX,18.0,0.000059


In [47]:
np.shape(train_df)

(307507, 799)

In [50]:
inf_columns

0       PREV_APP_CREDIT_PERC_MAX
1    REFUSED_APP_CREDIT_PERC_MAX
2        INSTAL_PAYMENT_PERC_MAX
dtype: object

In [ ]:
for column in inf_columns:
    inf_nb[i] = np.isinf(train_df[column]).sum()

    inf_ratio[i] = np.isinf(train_df[column]).sum()/len(train_df)
serie_inf_nb= serie_inf_nb.append(inf_nb,ignore_index = True)
serie_inf_ratio = serie_inf_ratio.append(inf_ratio,ignore_index =True)

#d = {'inf_columns':inf_columns,'inf_nb':serie_inf_nb,'inf_ratio':serie_inf_ratio}

inf_df = pd.concat([inf_columns,serie_inf_nb,serie_inf_ratio],axis=1)
inf_df.rename(columns={0:'inf_columns',1:'inf_nb',2:'inf_ratio'} )

In [32]:
nan_columns[1]

'AMT_GOODS_PRICE'

In [ ]:
type(len(train_df))

In [ ]:
nan_columns

In [44]:
np.isnan(train_df.loc[10,nan_columns[10]]).sum()

1

In [ ]:
np.isnan(train_df.loc[10,nan_columns[10]]).sum()

In [37]:
len(serie_nan_nb)

3

In [ ]:
CC_COUNT